In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

In [ ]:
###DESCRIPTIVES FOR ALL

directory = os.path.join(json_path_onedrive,
    'results', 'Avg_Features')

In [ ]:
tbl = pd.read_csv(os.path.join(avg_feat_path,'Sub005_AvgFeatures.csv'))

tbl

In [ ]:
##GOALS:
#1. Make function
#2. Save each time point table in it
#3. provide mean and std dictionary
#4. Make boxplots from these

df_fu0m = pd.DataFrame()
df_fu3m = pd.DataFrame()
df_fu12m = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, index_col=None)
        df = df.drop('Unnamed: 0', axis = 1)
        
        subID = filename[0:6]

        # Filter the data for 'FU0M', 'FU3M', and 'FU12M'
        fu0m_data = df[df['TimePoint'] == 'FU0M']
        fu3m_data = df[df['TimePoint'] == 'FU3M']
        fu12m_data = df[df['TimePoint'] == 'FU12M']
        
        fu0m_data.insert(0, 'SubID', subID)
        fu3m_data.insert(0, 'SubID', subID)
        fu12m_data.insert(0, 'SubID', subID)

        df_fu0m = pd.concat([df_fu0m, fu0m_data])
        df_fu3m = pd.concat([df_fu3m, fu3m_data])
        df_fu12m = pd.concat([df_fu12m, fu12m_data])


In [ ]:
df_fu3m

In [ ]:
# Calculate the means for all columns
means = this_df.mean()

# Divide 'Telemetry_AllSec' and 'TelemDurSecRes' by 60*2 and round to 2 decimal places
means['Telemetry_AllSec'] = (means['Telemetry_AllSec'] /  3600)
means['TelemDurSumSecRes'] = (means['TelemDurSumSecRes'] /  3600)
means['TelemDurSumSecWard'] = (means['TelemDurSumSecWard'] /  3600)
means['SensDurSumSec'] = (means['SensDurSumSec'] / 3600)

print(means.round(2))


In [ ]:
# Calculate the means for all columns
stds = this_df.std()

# Divide 'Telemetry_AllSec' and 'TelemDurSecRes' by 60*2 and round to 2 decimal places
stds['Telemetry_AllSec'] = (stds['Telemetry_AllSec'] /  3600)
stds['TelemDurSumSecRes'] = (stds['TelemDurSumSecRes'] /  3600)
stds['TelemDurSumSecWard'] = (stds['TelemDurSumSecWard'] /  3600)
stds['SensDurSumSec'] = (stds['SensDurSumSec'] / 3600)

print(stds.round(2))